In [1]:
from meshparty import trimesh_io, trimesh_vtk
from caveclient import CAVEclient
import numpy as np
import pandas as pd
import cloudvolume
import itkwidgets


#setting up the cave client, if you have not done this before checkout the notebook CAVEsetup.ipynb at 
#https://github.com/AllenInstitute/MicronsBinder/tree/master/notebooks/mm3_intro
client = CAVEclient()
#client.auth.get_new_token()
#client.auth.save_token(token="Your token here")


dataset_name = 'minnie65_public_v117'
client = CAVEclient(dataset_name)

seg_source = client.info.segmentation_source()

#using the google cloud segmentation path
cv = cloudvolume.CloudVolume("precomputed://gs://iarpa_microns/minnie/minnie65/seg", use_https=True)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
</b> Using the CAVEclient, we can query the synapse table for any neuron in the volume
      </div>

In [3]:
BC_ID = 864691135269913253
MC_ID = 864691135467660940

## Visualizing the cell alone

In [8]:
cellid = BC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])

In [9]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [10]:
cellid = MC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data])

In [11]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Showing both together

In [12]:
cellid = MC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data1 = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)
cellid = BC_ID
mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data2 = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

#setting up an itkwidgets viewer
viewer=itkwidgets.view(geometries=[poly_data1, poly_data2])

In [13]:
viewer

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Basket cell with synapses

In [71]:
cellid = BC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerBC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerBC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [70]:
cellid = MC_ID

mesh = cv.mesh.get(cellid,lod=3)[cellid]
poly_data = trimesh_vtk.trimesh_to_vtk(mesh.vertices, mesh.faces, None)

post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))

voxel_resolution = np.array([4,4,40])#setting the voxel resolution to convert to nm
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
viewerMC =itkwidgets.view(geometries=[poly_data],
                          point_sets = [syn_pts],
                          #point_sets_sizes=[syn_sizes],
                          point_sets_colors = ['r'])
viewerMC

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## More controlled rendering with `trimesh_vtk.render_actor`

In [60]:
cellid = MC_ID

# mesh
mesh = cv.mesh.get(cellid,lod=3)[cellid]
mesh_actors = trimesh_vtk.mesh_actor(mesh, color=(0, 0, 0.4))

# synapses
post_synapse_df = client.materialize.synapse_query(post_ids = np.array([cellid]))
syn_pts = np.vstack(post_synapse_df['ctr_pt_position'].values) * voxel_resolution
syn_sizes = np.array(post_synapse_df['size'].values)
syn_actors = trimesh_vtk.point_cloud_actor(syn_pts, size=syn_sizes/20, color=(1, 0, 0))
#                                           color=[() for i in range(len(syn_pts))])

camera = trimesh_vtk.camera_from_quat()
trimesh_vtk.render_actors([syn_actors, mesh_actors])

setting up renderer
done setting up
actors added
camera set
render done
finalizing..


<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLRenderer(0x5581b35e6970) at 0x7f0807eb3ac0>

In [69]:
trimesh_vtk.camera_from_quat?